<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---


<p align="center"><h1 align="center">Quick Start: Image Classification Tutorial with Custom Lambda </h1> 

---

<h3 align="center">(Deploy model to an AI Model Share Model Playground REST API<br> and Web Dashboard in five easy steps...)</h3></p>
<p align="center"><img width="100%" src="https://aimodelsharecontent.s3.amazonaws.com/aimstutorialsteps.gif" /></p>


---

In [ ]:
! pip install aimodelshare --upgrade


# **Deploying Your Own Model Using the aimodelshare Library**

> This tutorial will take you through how to deploy your own model on AWS's serverless computing infrastructure using AI Model Share Initiative's aimodelshare library. This tutorial consists of 5 stages:
> 1. [Train Model](#step-1)
> 2. [Prepare Deployment Folder](#step-2)
> 3. [Define Input And Output JSON Formats](#step-3)
> 4. [Write Custom Lambda Handler](#step-4)
> 5. [Deploy Model On AWS Using AI Model Share Initiative's aimodelshare Library](#step-5)
> 
> In the event that the user already has a repository containing all the files and folders pertaining to the inferencing given an input, Stages 1 and 2 can be omitted/done externally by the user. Users can then start the deployment process directly from Stage 3.

<a name="step-1"></a>
# **[1] Train Model**

> In this section, the user will build and train a model of their choice. Here, we train an image classification model. This will contain the following steps:
> 1. Importing All The Required Libraries
> 2. Downloading The Image Dataset And Storing Image Paths
> 3. Writing A Preprocessor Function For Preprocessing The Image Dataset
> 4. Preprocessing The Entire Dataset And Getting Corresponding Image Labels
> 5. Performing A Train Test Split
> 6. Training The Image Classification Model


### **[1.1] Importing All The Required Libraries**

> Import all the libraries necessary for preprocessing the dataset and training the image classification model.

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import tarfile
from six.moves import urllib

### **[1.2] Downloading The Image Dataset And Storing Image Paths**

> Download the flower image dataset from the TensorFlow repository and store the paths to each image.

In [3]:
# Downloading images from TensorFlow in a compressed .tgz file

directory = 'flower_photos' # Downloading to this directory drom the download URL.
download_url = 'http://download.tensorflow.org/example_images/flower_photos.tgz' # Downloading from this URL
categories = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips'] # categories of images

# If the images aren't already downloaded, download and save them to directory
def download_images(directory, download_url):
    if not os.path.exists(directory):
        print('Downloading images from from %s.' % download_url)
        urllib.request.urlretrieve(download_url, directory+'.tgz')
        tar = tarfile.open(directory+'.tgz', 'r')
        tar.extractall()
        tar.close()
    print('Images are located in %s.' % directory)

# Load file paths to fnames list object
def get_file_paths(directory, categories):
    fnames = []
    for category in categories:
        folder = os.path.join(directory, category)
        file_names = os.listdir(folder)
        full_path = [os.path.join(folder, file_name) for file_name in file_names]
        fnames.append(full_path)
    return fnames

download_images(directory, download_url)
fnames = get_file_paths(directory, categories)

Images are located in flower_photos.


### **[1.3] Writing A Preprocessor Function For Preprocessing The Image Dataset**

> Preprocessor is written to resize and normalize images using the cv2 library and then convert images to NumPy arrays.

In [4]:
def preprocessor(data, shape=(192, 192)):

    # Resize a color image and min/max transform the image
    img = cv2.imread(data) # read in image from filepath
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of BGR, we convert the order to RGB
    img = cv2.resize(img, shape) # change height and width of image
    img = img / 255.0 # min/max transform

    # Convert cv2 image to NumPy array
    X = np.array(img) # converting to NumPy array
    X = np.expand_dims(X, axis=0) # increase dimensions to convert object shape to [1, h, w, channels]
    X = np.array(X, dtype=np.float32) # reduce precision for faster training and inferencing

    return X

### **[1.4] Preprocessing The Entire Dataset And Getting Corresponding Image Labels**

> Loop over all the image file paths and preprocess the images using the defined preprocessor function and store the returned NumPy arrays.
> Store the labels of the images in a list with the index corresponding to the images in the sequence preprocessed.

In [5]:
# Prepare a multidimensional array containing all the images after passing them through a preprocessor function
def get_image_array(fnames):
    image_list = []
    for category in fnames:
        for f in category:
            image_list.append(preprocessor(f))
    return np.vstack(image_list)

# Prepare list of all the labels corresponding to the images in the multidimensional array
def get_image_labels(fnames, categories):
    y_labels = []
    for category_list, category in zip(fnames, categories):
        y_labels += [category] * len(category_list)
    return pd.get_dummies(y_labels)

X = get_image_array(fnames)
y = get_image_labels(fnames, categories)

### **[1.5] Performing A Train Test Split**

> Split the dataset into a train and test dataset to validate the model trained.

In [6]:
# train_test_split resized images
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.80, random_state = 1987)

### **[1.6] Training The Image Classification Model**

> Build a deep convolutional neural network for image classification using TensorFlow.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
                
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=[192, 192, 3]))
model.add(MaxPooling2D(pool_size=2))
   
model.add(Flatten())

model.add(Dense(len(categories), activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the network to the train set and validate with test set
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, verbose=1)

23/23 [==============================] - 29s 1s/step - loss: 15.0318 - accuracy: 0.2834 - val_loss: 8.1374 - val_accuracy: 0.3287


<a name="step-2"></a>
# **[2] Prepare Deployment Folder**

> In this section, the user will store all the files and folders related to the trained model in a single folder. This folder will be known as the deployment folder and will be uploaded to AWS's serverless computing infrastructure. The steps to prepare the deployment folder consist of:
> 1. Create Deployment Folder
> 2. Save Model Metadata And Weights To Deployment Folder
> 3. Write Inferencing Code And Save To Deployment Folder

### **[2.1] Create Deployment Folder**

> Create a new folder with a suitable name to store all the required files and folders in.

In [8]:
deployment_folder = 'deploy'

if os.path.isdir(deployment_folder):
    shutil.rmtree(deployment_folder)
os.mkdir(deployment_folder)

### **[2.2] Save Model Metadata And Weights To Deployment Folder**

> User has many pathways of saving model metadata and weights based on preference of how they will load model and weights during inferencing. Here, we use the aimodelshare library to convert TensorFlow Keras model to ONNX model and save as a file in the deployment folder.

In [9]:
# convert TensorFlow Keras model to ONNX model
from aimodelshare.aimsonnx import model_to_onnx
onnx_model = model_to_onnx(model,
                           framework='keras',
                           transfer_learning=False,
                           deep_learning=True)

# save the model to the deployment folder
with open(os.path.join(deployment_folder, "runtime_model.onnx"), "wb") as f:
    f.write(onnx_model.SerializeToString())

### **[2.3] Write Inferencing Code And Save To Deployment Folder**

> Inferencing code will:
> 1. Load the model into the global runtime environment
> 2. Preprocess the input passed
> 3. Pass preprocessed input to model and generate inference
> 4. Return inference to function call
>
> We save this inferencing code in the deployment folder. The custom lambda handler will call the inferencing function and pass desired inputs as the parameters to the function in their required format.


In [10]:
prediction_code = """

import onnxruntime as rt
import cv2
import numpy as np

# load the ML model in global environment for faster inference time for subsequent function calls
model_file_path = 'runtime_model.onnx'
model = rt.InferenceSession('./' + model_file_path)

# this is the preprocessor function 
def preprocessor(data, shape=(192, 192)):

    # Resize a color image and min/max transform the image
    img = cv2.imread(data) # read in image from filepath
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of BGR, we convert the order to RGB
    img = cv2.resize(img, shape) # change height and width of image
    img = img / 255.0 # min/max transform

    # Convert cv2 image to NumPy array
    X = np.array(img) # converting to NumPy array
    X = np.expand_dims(X, axis=0) # increase dimensions to convert object shape to [1, h, w, channels]
    X = np.array(X, dtype=np.float32) # reduce precision for faster training and inferencing

    return X

def predict(image_path):

    # labels corresponding to the index that is predicted
    labels = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

    # preprocessing the input data
    input_data = preprocessor(image_path)
    
    # generate prediction
    input_name = model.get_inputs()[0].name
    res = model.run(None, {input_name: input_data})
    prob = res[0]

    # map label to index with highest probability
    index = prob.argmax(axis=-1)
    result = list(map(lambda x: labels[x], index))

    return result

"""

In [11]:
with open(os.path.join(deployment_folder, "predict.py"), "w") as predict_file:
    predict_file.writelines(prediction_code)

Great! Now that we have our custom deployment folder ready, we will proceed with all the steps for creating a custom lambda handler that will convert all the inputs into formats required by the prediction model.

<a name="step-3"></a>
# **[3] Define Input And Output JSON Formats**

The input and output JSON structures should be specified by the user so that the specified input structure can be passed when the API calls are made and user can expect the specified output structure

This is the JSON structure template accessed using the 'body' key that the handler is expecting as specified by the user. Users can specify one or more key value pairs. There can be multiple types of values in the same JSON structure.

```
{
    "body": {
        "sample_key_1": "string",
        "sample_key_2": number,
        "sample_key_3": JSON object
    }
}
```

The JSON structure we are using here has 'data' as the key and its value is in the form of a string:

In [12]:
input_json_exampledata = {
         "data": "base64 encoded string"
    }

This is the list structure template returned to the 'body' key that the handler passes back to the function call. Users can return one or more values in the list. There can be multiple types of values in the same list.

```
{
    "body": ["string_1", integer_2, ["string_3", integer_4], ["string_5"], [integer_6]]
}
```

The list structure we are using here contains a list having just one string label:

In [13]:
output_json_exampledata = {
     ["label"]
}

<a name="step-4"></a>
# **[4] Write Custom Lambda Handler**

Now that we have defined our input and output JSON structures, we will write a custom lambda handler that will convert the specified JSON input into the formats required by the model for inferencing, and convert the result into the specified JSON output format.


### **Template:**

*Import all the required libraries.*

```
import numpy as np
import pandas as pd
```

*Import the name of the inference function to be called from the file with its relative path (the inference function 'predict_function' is accessed from a 'predict.py' file which is in a folder 'prediction_folder').*

```
from prediction_folder.predict import predict_funciton
```

*Now we define the handler (the function name must be 'handler' for the deployment to work).*

```
def handler(event, context):
```

*Contents of 'body' key is a string, we convert the content to a JSON object.*

```
    body = event['body']
    if isinstance(body, six.string_types):
        body = json.loads(body)
```

*Based on the user-specified JSON objects, we can access particular values using user-specified keys).*

```
    sample_string = body['sample_key_1']
    sample_number = body['sample_key_2']
    sample_json_object = body['sample_key_3]
```

*Now we convert all the values into the formats required by the model.*

```
    decoded_string = decode_string(sample_string)
    dataframe = dataframe_from_dict(sample_json_object)
```

*If the inference model requires path to the data instead of the actual data, save the data to the '/tmp' folder. Temporary data cannot be saved/stored anywhere else during the function call.*

```
    temp_string_file = "/tmp/temp_string_file"
    with open(temp_string_file, "w") as fh:
        fh.write(decoded_string)

    temp_dataframe_file = "/tmp/temp_dataframe_file"
    with open(temp_dataframe_file, "w") as fh:
        fh.write(dataframe)
```

*Pass all the required arguments to the imported inference function and store the output in a variable.*

```
    result = predict(temp_string_file, temp_dataframe_file)
```

*Convert the result back into format that the handler can serialize if required.*

```
    result = encode_string(result)
```

*Delete any temporary files created by the user that are needed for inferencing.*

```
    os.remove(temp_string_file)
    os.remove(temp_dataframe_file)
```

*Return the result in the acceptable format back to the handler call.*

```
    return result
```

### **Custom Lambda Handler Example For Image Classificaiton**

> This custom lambda handler does the following tasks:
> 1. Converting input string into JSON object
> 2. Fetching required values through corresponding keys
> 3. Converting values into the specified input data types for inferencing using the inference function
> 4. Converting values returned by inference function back into specified output data types
> 5. Return specified output values back to function call

In [14]:
custom_lambda_handler_code = """

# Importing all the required libraries
import numpy as np
import pandas as pd
import os
import six
import imghdr
import base64
import json

# Importing the predict function from the predict.py file
from predict import predict

def handler(event, context):

    # Contents of 'body' key is string, converting content to JSON object
    body = event['body']
    if isinstance(body, six.string_types):
        body = json.loads(body)

    # Fetching image in the form of a base64 encoded string stored within 'data' key of body
    data = body['data']

    # Decoding base64 encoded string into required data type
    sample = base64.b64decode(data)

    # Predict function requires path to image file as argument, saving data to file
    temp_file = "/tmp/temp_file"
    with open(temp_file, "wb") as fh:
        fh.write(sample)

    # Calling the  predict function
    result = predict(temp_file)

    # Deleting the image file created by the user for prediction
    os.remove(temp_file)

    # Returning the list with the label
    return result

"""

In [15]:
with open("custom_lambda_handler.py", "w") as handler_file:
    handler_file.writelines(custom_lambda_handler_code)

<a name="step-5"></a>
# **[5] Deploy Model On AWS Using AI Model Share Initiative's aimodelshare Library**

> We have finally finished modifying the deployment folder. Now we can use the aimodelshare library to deploy our model on AWS's serverless computing infrastructure and generate an API endpoint. To do so, we will be required to do the following:
> 1. Setting AWS Credentials
> 2. Deploying Model

### **[5.1] Setting AWS Credentials**

> The path to the user's AWS credentials, credentials.txt, should be passed as the parameter to set_credentials function.

In [16]:
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model", manual=False)

AI Model Share login credentials set successfully.
AWS credentials set successfully.


### **[5.2] Deploying Model**

> Once the credentials have been set, deploy_custom_lambda function should be called with the required parameters. Parameters:
> 1. input_json_exampldata: the format of the JSON input
> 2. output_json_exampldata: the format of the JSON output
> 3. custom_lambda_filepath: the relative path of the custom lambda handler file
> 4. deployment_dir: the deployment folder's relative path
> 5. custom_libraries: all the libraries required for running the model

In [ ]:
custom_lambda_filepath = "custom_lambda_handler.py"
deployment_dir = "deploy"
private = "FALSE"
custom_libraries = "onnxruntime,opencv-python,numpy,pandas,pillow"

from aimodelshare.deploy_custom_lambda import deploy_custom_lambda
deploy_custom_lambda(input_json_exampledata,
                     output_json_exampledata,
                     custom_lambda_filepath,
                     deployment_dir,
                     private,
                     custom_libraries=custom_libraries)

Test Lambda using this:

In [21]:
body={
      "data": "..."
  }


In [22]:
import requests
import json

data = json.dumps(body)
api_url = "your new rest api here"

headers = {"Content-Type": "application/json", "authorizationToken": "your token here" }

prediction = requests.request("POST", api_url, headers = headers, data=data)
print(prediction.text.encode("utf-8"))

b'["tulips"]'
